
<img src="https://datascientest.fr/train/assets/logo_datascientest.png" style="height:150px">

<hr style="border-width:2px;border-color:#75DFC1">
<h1 style = "text-align:center" > Introduction à TensorFlow </h1>
<h2 style = "text-align:center" > Couches et modèles personnalisés </h2>
<hr style="border-width:2px;border-color:#75DFC1">

> Les deux précédents exercices ont présenté quelques couches standards très utile en Deep learning (`Dense`, `Conv2D`...). Quelques fonctions de perte ou métriques prédéfinies ont été également utilisées.
>
> Il s'agit maintenant de personnaliser le tout: de la création de couches et de modèles, la définition des loss et métriques, jusqu'à l'entrainement et la prédiction.


## Couche personnalisée : Classe Layer


> La meilleure façon de créer ses propres couches personnalisées est de faire hériter la classe `Layer`. La classe devra également avoir au moins les 3 méthodes suivantes :
>
> * `__init__`: Initialise la classe `Layer` et définit les hyperparamètres de la couche.
>
>
> * `build` : Initialise les poids de la couche.
>
>
> * `call` : Applique l'opération sur l'entrée.
>
>
> Exemple pour la regression linéaire :
>
>```python
>from tensorflow.keras.layers import Layer
>
>class CustomLinear(Layer):
>     def __init__(self, units):
>         super(CustomLinear, self).__init__()
>         self.units = units
>
>     def build(self, input_shape):
>         self.w = self.add_weight(
>            shape = (input_shape[-1], self.units),
>            initializer = "random_normal",
>            trainable = True
>         )
>         self.b = self.add_weight(
>            shape = (self.units,), 
>            initializer = "random_normal",
>            trainable = True
>         )
>
>     def call(self, inputs):
>         return tf.matmul(inputs, self.w) + self.b
>```
>
> Une fois la couche définie, l'appel de l'instance va automatiquement initialiser les poids de la couche :
>
>```python
> # Définition d'une couche CustomLinear
>linear = CustomLinear()
> # Initialisation des poids ainsi que transforme l'entrée data
>linear(data)
>```
>
> Nous allons maintenant créer une couche représentant une régression polynomiale d'ordre 2 contenant des poids `w` et un biais `b`, cette couche va transformer les entrées en leur appliquant une combinaison linéaire avec les poids.

 
* Importer le module `tensorflow` sous le nom `tf`


* Importer la classe `Layer` de `tensorflow.keras.layers`

 
* Créer une classe **`CustomPolynomial`** qui hérite de la classe `Layer`. 
     * Dans le constructeur de cette classe `__init__` avec comme argument `self`, `units` :
         * Initialiser les objets de la classe de base `Layer`.
         * Ajouter l'attribut `units` à la classe.
     
     * Dans la méthode `build`, initialiser deux attributs $w$ et $b$ à l'aide de la méthode `add_weight`. Attention à bien multiplier la shape de w par deux (regression polynomiale d'ordre 2).
     
     * Dans l'appel `call` de la classe, concaténer l'entrée **inputs** avec **inputs^2**, puis appliquer une combinaison linéaire de ce résultat avec les poids `w` et qui rajoute un biais `b`.
     ```python
     def call(self, inputs):
         # inputs**2
         inputs_pow = tf.pow(inputs, 2)
         # Concatenation inputs and inputs**2
         inputs_process = tf.concat([inputs, inputs_pow], axis=-1)
         # Combinaison linéaire entre inputs_process et w
         return tf.matmul(inputs_process, self.w) + self.b
     ```

* Instancier un objet **`customlayer`** de la classe `CustomPolynomial` à 8 unités 


* Créer un tenseur **`inputs`** de dimensions (4,5) contenant que des 1.


* Appliquer la transformation de la couche `customlayer` à inputs.


* Afficher le résultat.

> La classe `Layer` permet de stocker les poids de chaque couche dans les <b>attributs</b> `weights`, `trainable_weights` et `non_trainable_weights` :
>
> * `weights` : L'ensemble des poids de la couche.
>
>
> * `trainable_weights` : Seulement les poids entrainables
>
>
> * `non_trainable_weights` : Seulement les poids non entrainables 

* Afficher le nombre de poids :
    * de la couche `customlayer`.
    
    * entrainable de la couche `customlayer`.
    
    * non entrainable de la couche `customlayer` .

number of weights:  2
number of trainable weights:  2
number of non trainable weights 0


> L'un des paramètres importants de la classe `Layer` est **`training`**, il permet de différencier le comportement du modèle en cas d'entraînement ou d'inférence (prédiction). Cette notion est notamment important dans les couches de `Dropout` et de `BatchNormalization`. 


* Créer une couche personnalisée `CustomDropout` qui initialise un attribut `rate`.


* Dans l'appel `call` de la classe avec comme argument `inputs` et `training = None` :
    * S'il y a <b>entrainement</b>, retourne un dropout des entrées à l'aide de la fonction `dropout` de `tf.nn` (en paramètre les entrées et le taux `rate`).

    * S'il n'y a <b>pas d'entrainement</b>, retourne les entrées.

## La classe Model

> Grace à la classe `Layer`, il est très facile de créer des couches personnalisées compatibles avec le constructeur `Sequential`. Il est également possible de créer, de la même façon, un modèle personnalisé en faisant hériter cette fois la classe `Model`.
>
> L'initialisation du modèle permet de définir toutes les couches nécessaires en attributs. L'appel du modèle, quant à lui, permet de passer nos entrées à travers les différentes couches du modèle :
>
> ```python
>from tensorflow.keras.models import Model
>from tensorflow.keras.layers import Softmax
>
>class CustomModel(Model):
>     def __init__(self, units_1, units_2, n_classes):
>         # Initialisation de la classe Model.
>         super(CustomModel, self).__init__()
>         # Définition de chaque couche de notre modèle.
>         self.linear_1 = CustomLinear(units_1)
>         self.dropout_1 = CustomDropout(rate=0.2)
>        
>         self.linear_2 = CustomLinear(units_2)
>         self.dropout_2 = CustomDropout(rate=0.2)
>        
>         self.linear_3 = CustomLinear(n_classes)
>         self.softmax = Softmax()
>
>     def call(self, inputs):
>         # Appliquer l'opération de chaque couche.
>         x = self.linear_1(inputs)
>         x = tf.nn.relu(x)
>         x = self.dropout_1(x)
>        
>         x = self.linear_2(x)
>         x = tf.nn.relu(x)
>         x = self.dropout_2(x)
>        
>         x = self.linear_3(x)
>         return self.softmax(x)
>
>```
>
> L'avantage d'utiliser la classe **`Model`** par rapport à la classe **`Layer`** :
>
>
>>* Possède les méthodes suivantes : `fit`, `evaluate`, `predict`.
>>
>>
>>* Contient ses propres couches internes, accessibles à l'aide de `model.layers`.
>>
>>
>>* Permet de sauvegarder et sérialiser : `save_weights`, `load_weights`.

Ici, nous pouvons nous contenter du constructeur `Sequential` :

* Définir un modèle par couche vierge **model** à l'aide du constructeur `Sequential`.


* Ajouter une couche `CustomPolynomial` avec 32 units.


* Ajouter une couche `CustomDropout` avec 0.2 comme rate.


* Ajouter une couche d'activation `ReLU` de `tensorflow.keras.layers`.


* Ajouter une couche `CustomPolynomial` avec 64 units.


* Ajouter une couche `CustomDropout` avec 0.2 comme rate.


* Ajouter une couche d'activation `ReLU` de `tensorflow.keras.layers`.


* Ajouter une couche `CustomPolynomial` avec 1 unit.


* Ajouter une couche d'activation `ReLU` de `tensorflow.keras.layers`.

* Afficher le résultat que retourne le modèle pour un vecteur de 1 de taille [10,5].

> Nous allons maintenant traiter un problème de régresion avec notre modèle personnalisé fraichement créé.

* Charger le jeu de données <i>"airfoil_data.csv"</i> dans le dataframe **df**.

In [ ]:
import pandas as pd
df = pd.read_csv('airfoil_data.csv')
df.head()

* Séparer les données en **`features`** et **`target`** sachant que la variable cible est "PressureLevel".


* Séparer les données en un ensemble d'entrainement (**X_train**, **y_train**) et en un ensemble de test (**X_test**, **y_test**). On peut prendre une proportion de 20% pour l'ensemble de test.

* Exécuter la cellule suivante pour normaliser nos données.

In [45]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)

y_train = y_train.ravel().reshape(-1, 1)
y_test = y_test.ravel().reshape(-1, 1)
y_scaler = StandardScaler().fit(y_train)

# Features
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

# Target
y_train = y_scaler.transform(y_train)
y_test = y_scaler.transform(y_test)

* Compiler notre modèle **model** avec un optimizer `Adam`, une fonction de perte `MeanSquaredError` et une métrique `MeanAbsoluteError`.

* Entraîner le modèle sur `X_train` et `y_train`.

<hr style="border-width:2px;border-color:#75DFC1">
<h2 style = "text-align:center" > Ce qu'il faut retenir </h2> 
<hr style="border-width:2px;border-color:#75DFC1">


> Il est ainsi possible de créer des couches et des modèles personnalisés tout en profitant de l'API de keras qui fournit des méthodes utiles à la modélisation et l'entrainement.
>
> Beaucoup de modèle nécessite d'utilise des couches personnalisées ou une structure bien particulière. C'est notamment le cas pour les modèles célèbres suivants:
>
> * Seq2seq : modèle sequence-to-sequence
>
>
> * Transformer : modèle de BERT, modèle CamemBERT, XLNet ...
>
>
> * WaveNet : modèle basé sur des convolutions 1D.
>
>
> * Spatial Transformer Networks : Remplacer la couche de pooling (https://kevinzakka.github.io/2017/01/18/stn-part2/)
>
>
> Il est aussi possible d'accéder à d'avantages de couches/modèles prédéfinies sur [TensorFlow Hub](https://www.tensorflow.org/hub?hl=fr).